In [ ]:
from typing import Any, Optional
import numpy as np
import os
import pymongo
import pyarrow as pa
import ray
import pandas as pd
from openai import OpenAI
from pyarrow import csv
from pymongo import MongoClient, ASCENDING, DESCENDING, UpdateOne
from pymongo.operations import SearchIndexModel, IndexModel

from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from ray.util.accelerators import NVIDIA_TESLA_A10G

In [11]:
from accelerate import  Accelerator
import torch
torch.cuda.empty_cache()
Accelerator().free_memory()

[]

In [1]:
import argparse
import os
import subprocess

import torch
from PIL import Image

from vllm import LLM
from vllm.multimodal.image import ImageFeatureData, ImagePixelData

2024-06-25 23:59:41,736	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.2 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# llm = LLM(
#     model="llava-hf/llava-1.5-7b-hf",
#     image_input_type="pixel_values",
#     image_token_id=32000,
#     image_input_shape="1,3,336,336",
#     image_feature_size=576,
#     disable_image_processor=False,
# )
llm = LLM(
    model="llava-hf/llava-v1.6-mistral-7b-hf",
    # image_input_type="pixel_values",
    # image_token_id=32000,
    # image_input_shape="1,3,336,336",
    # image_feature_size=576,
    # disable_image_processor=False,
    **{
        "trust_remote_code": True,
        "enable_lora": False,
        "max_num_seqs": 1,
        "max_model_len": 14224,
        "gpu_memory_utilization": 0.85,
        "image_input_type": "pixel_values",
        "image_token_id": 32000,
        "image_input_shape": "1,3,336,336",
        "image_feature_size": 1176,
        "enforce_eager": True,
    }
)

INFO 06-25 23:59:42 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='llava-hf/llava-v1.6-mistral-7b-hf', speculative_config=None, tokenizer='llava-hf/llava-v1.6-mistral-7b-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=14224, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=llava-hf/llava-v1.6-mistral-7b-hf)


/home/ray/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-25 23:59:44 weight_utils.py:218] Using model weights format ['*.safetensors']
INFO 06-25 23:59:52 model_runner.py:160] Loading model weights took 14.1020 GB
INFO 06-25 23:59:57 gpu_executor.py:83] # GPU blocks: 1316, # CPU blocks: 2048


In [3]:
disable_image_processor=False

prompt = "<image>" * 1176 + (
    "\nUSER: What is the content of this image?\nASSISTANT:")

if disable_image_processor:
    image = torch.load("images/stop_sign_pixel_values.pt")
else:
    image = Image.open("stop_sign.jpg")


In [4]:
outputs = llm.generate({
    "prompt": prompt,
    "multi_modal_data": ImagePixelData(image),
})


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 06-26 00:00:04 llava_next.py:98] Dynamic image shape is currently not supported. Resizing input image to (336, 336).


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.10it/s, est. speed input: 1316.36 toks/s, output: 17.64 toks/s]


In [5]:
outputs[0].outputs

[CompletionOutput(index=0, text=' The image shows a red stop sign with the word "STOP" written in white', token_ids=[415, 3469, 4370, 264, 2760, 2115, 1492, 395, 272, 1707, 345, 26599, 28739, 4241, 297, 3075], cumulative_logprob=-4.296915684157284, logprobs=None, finish_reason=length, stop_reason=None)]

In [ ]:
class MistralvLLM:
    def __init__(self):
        self.llm = LLM(
            model="mistralai/Mistral-7B-Instruct-v0.1",
            max_model_len=4096,
            skip_tokenizer_init=True,
        )


In [ ]:
llm = MistralvLLM()

In [ ]:
sampling_params = SamplingParams(
    n=1,
    presence_penalty=0,
    frequency_penalty=0,
    repetition_penalty=1,
    length_penalty=1,
    top_p=1,
    top_k=-1,
    temperature=0,
    use_beam_search=False,
    ignore_eos=False,
    max_tokens=2048,
    seed=None,
    detokenize=False,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
            "mistralai/Mistral-7B-Instruct-v0.1",
        )
input_text = "What is the capital of france?"
input_tokens = tokenizer.apply_chat_template(
    conversation=[{"role": "user", "content": input_text}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="np",
).squeeze()
input_tokens

In [ ]:
response = llm.llm.generate(prompt_token_ids=input_tokens.tolist(), sampling_params=sampling_params)

In [ ]:
response

In [ ]:
response[0].outputs[0].token_ids

In [ ]:
tokenizer.decode(response[0].outputs[0].token_ids)